# Important Pandas Functions

Adapted from Wes McKinney's Python for Data Analysis and the Pandas Documentation

In [ ]:
import numpy as np
import pandas as pd

# Reindexing

Reindexing takes a current Pandas object and creates a *new* Pandas object that *conforms* to the specified index:

In [ ]:
original = pd.Series([1.4, 2.3, 3.1, 4.2], index = ['d','c','a','b'])

In [ ]:
original

In [ ]:
newobj = original.reindex(['a','b','c','d','e'])

In [ ]:
newobj  # takes the data in orignal and moves it so it conforms to the specified index
# values that do not exist for the new index get NaN

In [ ]:
# if you don't want NaN, you can specify a fill_value
newobj2 = original.reindex(['a','b','c','d','e'], fill_value = 0)
newobj2

For ordered data like a time series, it might be desirable to fill values when reindexing

In [ ]:
obj3 = pd.Series(['blue', 'purple', 'yellow'], index=[0, 3, 6])
obj3

In [ ]:
obj3.reindex(range(9), method='ffill')  
# forward-fill pushes values 'forward' until a new value is encountered

In [ ]:
obj3.reindex(range(9), method='bfill')  
# back-fill works in the opposite direction
# there was no value at index 8 so, NaNs get filled in

In [ ]:
# we specify the creation of a date_index using the date_range function
# freq = 'D' creates Daily values 
date_index = pd.date_range('1/1/2010', periods=6, freq='D')
date_index

In [ ]:
# we create a DataFrame with the date index
df2 = pd.DataFrame({"prices": [100, 101, np.nan, 100, 89, 88]}, index=date_index)
df2

In [ ]:
date_index2 = pd.date_range('12/29/2009', periods=10, freq='D')  # a new date index
df2.reindex(date_index2)

## `.reindex()` vs `.loc()`

If you don't need to fill in any missing info, then .reindex() and .loc() work the same
If the new index will have values that don't exist in the current index, you need to use reindex

In [ ]:
obj5 = pd.DataFrame({'val':[1.4, 2.3, 3.1, 4.2]}, index = ['d','c','a','b'])
obj5

In [ ]:
obj5.reindex(['a','b','c','d'])

In [ ]:
obj5.loc[['a','b','c','d']] # works the same as reindex

In [ ]:
obj5.reindex(['a','b','c','d','e'])

In [ ]:
obj5.loc[['a','b','c','d','e']]  # .loc() returns a warning or error if you give an entry in the index that doesn't exist

# Dropping rows or columns

you can use `df.drop()` to remove rows (default) or columns (specify axis = 1) at certain index locations.

In [ ]:
df = pd.DataFrame(np.arange(12).reshape(3,4), columns=['A', 'B', 'C', 'D'], index = ['a','b','c'])
df

In [ ]:
# drop columns
df.drop(['B', 'C'], axis=1)

In [ ]:
# drop rows
df.drop(['a', 'c'])

In [ ]:
# df.drop returns a new object and leaves df unchanged
# you can change this behavior with the argument inplace = True
df

# Data Alignment

When performing element-wise arithmetic, Pandas will align the index values before doing the computation

In [ ]:
s1 = pd.Series([7.3, -2.5, 3.4, 1.5], index=['a', 'c', 'd', 'e'])
s1

In [ ]:
s2 = pd.Series([-2.1, 3.6, -1.5, 4, 3.1],
               index=['a', 'c', 'e', 'f', 'g'])
s2

In [ ]:
s1 + s2

For data frames with different columns, the rows and columns will be aligned

In [ ]:
df1 = pd.DataFrame(np.arange(9.).reshape((3, 3)), columns=list('bcd'),
                   index=['Ohio', 'Texas', 'Colorado'])
df1

In [ ]:
df2 = pd.DataFrame(np.arange(12.).reshape((4, 3)), columns=list('bde'),
                   index=['Utah', 'Ohio', 'Texas', 'Oregon'])
df2

In [ ]:
df1 + df2 
# c is in df1, but not df2
# e is in df2, but not df1
# the result returns the union of columns, but will fill in NaN for elements that do not exist in both

In [ ]:
# if you want to fill in values that are missing, you can use df.add() and specify the fill_value
# this will perform the above operation, but instead of using NaN when it can't find a value (which will return NaN),
# it will use the fill_value
df1.add(df2, fill_value = 0)
# you still get NaN if the value does not exist in either DataFrame

In [ ]:
# other arithmetic operations that can be called on DataFrames are:
# .add()
# .sub()
# .mul()
# .div()
# .floordiv()

# Summary Stats of a DataFrame

In [ ]:
df = pd.DataFrame({'one':[1.5,6.0,np.nan, 1.5,4,6, np.nan],
                   'two':[np.nan, -4.5, np.nan, -1.5, 0, -4.5, 4]},
                  index=['a', 'b', 'c', 'd','e','f','g'])
df

In [ ]:
df.sum()  # default behavior skips missing values

In [ ]:
df.sum(skipna = False)

In [ ]:
df.mean()

In [ ]:
df.idxmin()  # which row has the minimum value, also .idxmax()
# returns the first minimum, if there are multiple

In [ ]:
df.one.unique()  # shows the unique values in the order observed

In [ ]:
df.one.nunique()  # number of non-missing unique values exist

In [ ]:
df.one.value_counts()  # tally up counts of each value
# returns a series. the index are the unique values observed, the values are the frequencies.
# they appear in descending order of frequency

In [ ]:
df.one.isin([1.5, 4.0]) # checks to see if the value has membership in a particular list
# returns a series with boolean values

In [ ]:
df.loc[  df.one.isin([1.5,4.0]),  ]  # can filter rows based on the .isin() membership

# filtering out missing values

In [ ]:
df

In [ ]:
df.dropna() # gets rid of any row that is not complete

In [ ]:
df.dropna(how = 'all')  # only drops rows that are entirely NaN

In [ ]:
# you can also use .notnull(), which is True for values that are not missing
df[df.two.notnull()]  # You can use this in conjuntion with specifying the column

# Filling in Missing Values

In [ ]:
df

In [ ]:
df.fillna(0) # fill in missing values with a constant

In [ ]:
df.fillna({'one': 1000, 'two': 0})  # use a dictionary to specify values to use for each column

In [ ]:
df.fillna(method = 'bfill')  # backfills. You can also use ffill

In [ ]:
df.fillna(df.mean())  # fill na with df.mean() will fill in the column means

all of the above fillna methods have created new DataFrame objects. If you want to modify the current DataFrame, you can use the optional argument `inplace = True`

# dealing with duplicates

In [ ]:
df

In [ ]:
df.duplicated()  # sees if any of the rows are a duplicate of an earlier row